In [ ]:
import subprocess
import math
from pydub import AudioSegment

def extract_audio_from_video(video_path, audio_path):
    command = [
        "ffmpeg",
        "-i",
        video_path,
        "-vn",
        audio_path,
    ]
    subprocess.run(command)

def cut_audio_in_chunks(audio_path, chunk_size, chunks_folder):
    track = AudioSegment.from_mp3(audio_path)
    chunk_overlap = 10 * 1000 # overlap_size = 10 seconds
    chunk_len = chunk_size * 60 * 1000 - chunk_overlap
    chunks = math.ceil(len(track) / chunk_len)
    for i in range(chunks):
        start_time = i * chunk_len
        end_time = (i+1) * chunk_len + chunk_overlap
        chunk = track[start_time:end_time]
        chunk.export(f"./{chunks_folder}/chunk_{i}.mp3", format="mp3")

cut_audio_in_chunks("./files/Bible_audio.mp3", 10, "./files/chunks")

In [4]:
import openai
import glob

def transcribe_chunks(chunk_folder, destination):
    files = glob.glob(f"{chunk_folder}/*.mp3")
    final_transcript = ""
    for file in files:
        with open(file, "rb") as audio_file, open(destination, "a") as text_file:
            transcipts = openai.Audio.transcribe(
                "whisper-1",
                audio_file,
                language="ko",
            )
            text_file.write(transcipts["text"])

transcribe_chunks("./files/chunks", "./files/chunks/transcipts.txt")